In [1]:
import pandas as pd 
import os 

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import seaborn as sns

from datasets import load_dataset
import numpy as np

In [4]:
cd /home/cn356/oliver/research/perturb-r

/home/cn356/oliver/research/perturb-r


# Utils

In [21]:
def chunking(model_response, granularity=20):
    """Split thinking into chunks of approximately equal size."""
    # Step 1: merge small chunks
    thinking = model_response.split('</think>')[0]
    chunks = thinking.split('\n\n')
    masks = [len(chunk.split()) > granularity for chunk in chunks]

    merged, buffer = [], []
    for c, m in zip(chunks, masks):
        if not m:
            buffer.append(c)
        else:
            if buffer:
                merged.append('\n\n'.join(buffer))  # Use '\n\n' to maintain paragraph structure
                buffer.clear()
            merged.append(c)
    if buffer:
        merged.append('\n\n'.join(buffer))

    # Step 2: merge small chunks to big chunks
    super_chunks, current = [], None
    for c in merged:
        if len(c.split()) > granularity:
            if current is not None:
                super_chunks.append(current)
            current = c
        else:
            if current is None:
                # no big chunk yet
                current = c
            else:
                current += '\n\n' + c  # Use '\n\n' to maintain paragraph structure

    if current is not None:
        super_chunks.append(current)

    return super_chunks

# Reasoning Results

In [5]:
root = "./results/hendrycks_math/sample200/reasoning"
os.listdir(root)

['QwQ-32B.pickle',
 'QwQ-32B_correct.pickle',
 'Qwen-R1-Distill-32B.pickle',
 'Qwen-R1-Distill-32B_correct.pickle']

In [6]:
for fname in os.listdir(root):
    if 'correct' not in fname:
        print(fname)
        df = pd.read_pickle(os.path.join(root, fname))
        print(f"Accuracy: {df['correct'].mean()}")

QwQ-32B.pickle
Accuracy: 0.97
Qwen-R1-Distill-32B.pickle
Accuracy: 0.955


In [7]:
df = pd.read_pickle(os.path.join(root, "QwQ-32B.pickle"))

In [17]:
response = df["response"][0]

In [23]:
len(chunking(response))

32

# Digit Corruption Results

In [42]:
def group_analysis (df: pd.DataFrame, cutoffs: list, col: str):
    def group(col, cutoffs = cutoffs):
        for cutoff in cutoffs:
            if col< cutoff:
                return cutoff
            else:
                return np.float("inf")
    
    df['token_count_groups'] = df[col].apply(group)
    
    return {"corrupting token counts": df['token_count_groups'].value_counts(), 
            "accuracy": df.groupby('token_count_groups')[col].mean()}

### Corrupt answer digit backwards

In [172]:
rl_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/Qwen2.5-Math-1.5B-DrGRPO-Zero_type=answer_digit_continuethinking=False.pickle")

In [173]:
corruption_group_analysis(rl_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 0.50    31
 1.00    27
 0.25    21
 0.75    19
 0.10    17
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.882353
 0.25    0.857143
 0.50    0.741935
 0.75    0.684211
 1.00    0.740741
 Name: still_correct, dtype: float64}

In [149]:
print(rl_df['post_corruption_response'][6])

 I started by recalling the definition of a complex number, which is \( z = x + yi \), where \( x \) and \( y \) are real numbers, and \( i \) is the imaginary unit. The conjugate of \( z \), denoted \( \bar{z} \), is \( x - yi \). 

I then substituted \( z = x + yi \) and \( \bar{z} = x - yi \) into the given equation \( 3z + 5i\bar{z} = 1 - 8i \). Expanding both terms, I got \( 3x + 3yi + 5ix - 5y = (3x + 5y) + (3y + 5x)i \). 

Setting the real and imaginary parts equal to the corresponding parts on the right-hand side, I obtained the system of linear equations: \( 3x + 5y = 1 \) and \( 5x + 3y = -8 \). 

To solve this system, I used the method of elimination. I multiplied the first equation by 5 and the second equation by 3 to align the coefficients of \( x \), resulting in the equations \( 12x + 16y = 5 \) and \( 12x + 9y = -25 \). Subtracting the second equation from the first, I found \( 7y = 28 \), leading to \( y = 5 \). Substituting \( y = 5 \) back into the first equation, I 

In [91]:
rl_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/R1-Distill-Qwen-1.5B_type=answer_digit_continuethinking=False.pickle")

In [92]:
corruption_group_analysis(rl_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 0.50    298
 0.25    290
 1.00    153
 0.75    151
 0.10    143
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.335664
 0.25    0.082759
 0.50    0.097315
 0.75    0.079470
 1.00    0.104575
 Name: still_correct, dtype: float64}

### Corrupt all digits forwards

In [174]:
sft_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/Qwen2.5-Math-1.5B-DrGRPO-Zero_type=midway_continuethinking=True.pickle")

In [175]:
corruption_group_analysis(sft_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 1.00    137
 0.50     71
 0.75     44
 0.25     27
 0.10      2
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.500000
 0.25    0.407407
 0.50    0.183099
 0.75    0.136364
 1.00    0.138686
 Name: still_correct, dtype: float64}

In [152]:
rl_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/Qwen2.5-Math-1.5B-sky-math8k-r1-continue-sft64steps-rl300steps_type=midway_continuethinking=True.pickle")

In [153]:
corruption_group_analysis(rl_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 1.00    106
 0.50     79
 0.75     65
 0.25     43
 0.10      4
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.250000
 0.25    0.302326
 0.50    0.113924
 0.75    0.107692
 1.00    0.216981
 Name: still_correct, dtype: float64}

# Distraction

## Full distraction

In [5]:
ls

config/                  llm_engine.py     results/
data/                    notebook/         reward_score/
digit_corruption.py      __pycache__/      run_reasoner.py
distract_thinking.py     random/           scripts/
environment_full.yml     README.md
environment_minimal.yml  requirements.txt


In [6]:
from reward_score.math import last_boxed_only_string, remove_boxed

def if_answer (response):
    try:
        math_answer = remove_boxed(last_boxed_only_string(response))
        return True
    except:
        return False
    
def group_analysis (df: pd.DataFrame, cutoffs: list, col: str, stats: str):
    cutoffs.sort()
    
    def group(col, cutoffs = cutoffs):        
        for cutoff in cutoffs:
            if col < cutoff:
                return f"< {cutoff}"

        return f"> {cutoffs[-1]}"

    df["answer_rate"] = df["post_distraction_response"].apply(if_answer)
    df['token_count_groups'] = df[col].apply(group)
    
    token_dist = df['token_count_groups'].value_counts()
    perf = df.groupby('token_count_groups')[stats].mean().loc[token_dist.index]
    answer_rate = df.groupby('token_count_groups')["answer_rate"].mean().loc[token_dist.index]
    
    return {"corrupting token counts": token_dist, 
            "answer rate": answer_rate,
            "accuracy": perf
           }

In [8]:
root = './results/hendrycks_math/sample200/distract_thinking'
cutoffs = [512, 1024, 2048, 4096]
os.listdir(root)

['R1-Distill-Qwen-1.5B.pickle',
 'R1-Distill-Qwen-7B.pickle',
 'R1-Distill-Qwen-14B.pickle',
 'QwQ-32B.pickle',
 'R1-Distill-Llama-8B.pickle',
 'Qwen-R1-Distill-32B.pickle']

In [22]:
df = pd.read_pickle(os.path.join(root, 'R1-Distill-Qwen-1.5B.pickle'))

print("accuracy according to original solution: \n", group_analysis(df, cutoffs, "num_distract_tokens", "original_correct"))
print('*' * 50, '\n\n')
print("accuracy according to distractor solution: \n", group_analysis(df, cutoffs, "num_distract_tokens", "distractor_correct"))

accuracy according to original solution: 
 {'corrupting token counts': token_count_groups
< 512     268
< 1024    176
< 2048    165
< 4096    107
> 4096     64
Name: count, dtype: int64, 'answer rate': token_count_groups
< 512     0.932836
< 1024    0.948864
< 2048    0.993939
< 4096    0.925234
> 4096    0.921875
Name: answer_rate, dtype: float64, 'accuracy': token_count_groups
< 512     0.667910
< 1024    0.602273
< 2048    0.612121
< 4096    0.411215
> 4096    0.093750
Name: original_correct, dtype: float64}
************************************************** 


accuracy according to distractor solution: 
 {'corrupting token counts': token_count_groups
< 512     268
< 1024    176
< 2048    165
< 4096    107
> 4096     64
Name: count, dtype: int64, 'answer rate': token_count_groups
< 512     0.932836
< 1024    0.948864
< 2048    0.993939
< 4096    0.925234
> 4096    0.921875
Name: answer_rate, dtype: float64, 'accuracy': token_count_groups
< 512     0.123134
< 1024    0.227273
< 2048 

In [29]:
df["distract_reasoning"][0]

"Okay, so I need to find the slope of a line that's perpendicular to another line. The given line has the equation \\(\\frac{x}{4} - \\frac{y}{5} = 1\\). Hmm, I remember that perpendicular lines have slopes that are negative reciprocals of each other. So, if I can find the slope of the given line, then I can just flip it and change the sign to get the perpendicular slope."

In [26]:
print(df["post_distraction_response"][0])

 Let me try that.

First, I need to rewrite the equation \(\frac{x}{4} - \frac{y}{5} = 1\) into slope-intercept form, which is \(y = mx + b\). That way, I can easily identify the slope \(m\).

Starting with \(\frac{x}{4} - \frac{y}{5} = 1\), I can multiply both sides by 20 to eliminate the denominators. Multiplying each term by 20 gives me \(5x - 4y = 20\). Now, I'll solve for \(y\):

\(5x - 4y = 20\)

Subtract \(5x\) from both sides:

\(-4y = -5x + 20\)

Now, divide every term by -4:

\(y = \frac{5}{4}x - 5\)

So, the slope \(m\) of the given line is \(\frac{5}{4}\). Since the line we're looking for is perpendicular, its slope \(m'\) should be the negative reciprocal of \(\frac{5}{4}\), which is \(-\frac{4}{5}\). That makes sense because \(\frac{5}{4} \times -\frac{4}{5} = -1\), satisfying the perpendicular condition.

Alright, so the slope of the perpendicular line is \(-\frac{4}{5}\). But wait, the question is about the area ratio between triangle AXY and triangle ABC. I don't immed

In [20]:
df = pd.read_pickle(os.path.join(root, 'R1-Distill-Qwen-7B.pickle'))

print("accuracy according to original solution: \n", group_analysis(df, cutoffs, "num_distract_tokens", "original_correct"))
print('*' * 50, '\n\n')
print("accuracy according to distractor solution: \n", group_analysis(df, cutoffs, "num_distract_tokens", "distractor_correct"))

accuracy according to original solution: 
 {'corrupting token counts': token_count_groups
< 512     315
< 2048    284
< 1024    202
> 4096     54
< 4096     45
Name: count, dtype: int64, 'answer rate': token_count_groups
< 512     0.990476
< 2048    0.992958
< 1024    0.990099
> 4096    1.000000
< 4096    0.977778
Name: answer_rate, dtype: float64, 'accuracy': token_count_groups
< 512     0.368254
< 2048    0.334507
< 1024    0.356436
> 4096    0.333333
< 4096    0.444444
Name: original_correct, dtype: float64}
************************************************** 


accuracy according to distractor solution: 
 {'corrupting token counts': token_count_groups
< 512     315
< 2048    284
< 1024    202
> 4096     54
< 4096     45
Name: count, dtype: int64, 'answer rate': token_count_groups
< 512     0.990476
< 2048    0.992958
< 1024    0.990099
> 4096    1.000000
< 4096    0.977778
Name: answer_rate, dtype: float64, 'accuracy': token_count_groups
< 512     0.584127
< 2048    0.612676
< 1024 

In [97]:
example = df.sample(n=1).iloc[0]
print("Original Correct?: ", example["original_correct"], "Distractor Correct?: ", example["distractor_correct"])
print(example["problem"], "\n\n", "=" * 100)
print(example["distract_reasoning"], "\n\n", "=" * 100)
print(example["post_distraction_response"])

Original Correct?:  1.0 Distractor Correct?:  0.0
<｜begin▁of▁sentence｜><｜User｜>In triangle $ABC,$ $\overline{CD}$ is the bisector of angle $C,$ with $D$ on $\overline{AB}.$  If $\cos \frac{C}{2} = \frac{1}{3}$ and $CD = 6,$ compute
\[\frac{1}{BC} + \frac{1}{AC}.\]<｜Assistant｜><think>
 

Alright, so I have this problem here about projections onto vectors that are orthogonal. Let me try to figure it out step by step.

First, the problem says that vectors a and b are orthogonal. Hmm, okay, so that means their dot product is zero. I remember that projections of a vector onto another vector can be found using the formula:

proj_u v = ( (u ⋅ v) / ||u||² ) * u

Where u is the vector onto which we're projecting, and v is the vector we're projecting onto. So, in this case, we're given that the projection of the vector [0, 13] onto a is [6, 4]. Let me write that down:

proj_a [0; 13] = [6; 4]

And we need to find the projection of the same vector [0; 13] onto b, which is orthogonal to a.

Since 

In [98]:
print("Original Solution: ", example["solution"], "\n\n", "=" * 100)
print("Distractor Solution: ", example["distract_solution"])

Original Solution:  The area of triangle $ABC$ is given by
\[[ABC] = \frac{1}{2} AC \cdot BC \cdot \sin C.\][asy]
unitsize (1 cm);

pair A, B, C, D;

A = (0,0);
B = (5,0);
C = (1,2);
D = extension(C, incenter(A,B,C), A, B);

draw(A--B--C--cycle);
draw(C--D);

label("$A$", A, SW);
label("$B$", B, SE);
label("$C$", C, N);
label("$D$", D, S);
label("$6$", (C + D)/2, NE);
[/asy]

We can also write
\begin{align*}
[ABC] &= [ACD] + [BCD] \\
&= \frac{1}{2} AC \cdot CD \sin \frac{C}{2} + \frac{1}{2} BC \cdot CD \sin \frac{C}{2} \\
&= 3AC \sin \frac{C}{2} + 3BC \sin \frac{C}{2} \\
&= 3 (AC + BC) \sin \frac{C}{2}.
\end{align*}Thus,
\[\frac{1}{2} AC \cdot BC \cdot \sin C = 3(AC + BC) \sin \frac{C}{2}.\]Then
\[AC \cdot BC \sin \frac{C}{2} \cos \frac{C}{2} = 3(AC + BC) \sin \frac{C}{2},\]so
\[\frac{AC \cdot BC}{3} = 3 (AC + BC).\]Hence,
\[\frac{1}{AC} + \frac{1}{BC} = \frac{AC + BC}{AC \cdot BC} = \boxed{\frac{1}{9}}.\] 

Distractor Solution:  Since $\begin{pmatrix} 6 \\ 4 \end{pmatrix}$ is the proj

<｜begin▁of▁sentence｜><｜User｜>In triangle $ABC,$ $\overline{CD}$ is the bisector of angle $C,$ with $D$ on $\overline{AB}.$  If $\cos \frac{C}{2} = \frac{1}{3}$ and $CD = 6,$ compute
\[\frac{1}{BC} + \frac{1}{AC}.\]<｜Assistant｜><think>
Alright, so I have this geometry problem here. Let me read it again: In triangle ABC, CD is the bisector of angle C, with D on AB. We're given that cos(C/2) = 1/3 and CD = 6. We need to compute 1/BC + 1/AC.

Hmm, okay. So, triangle ABC, with angle C being bisected by CD, where D is on AB. We have some trigonometric information about angle C and the length of the angle bisector CD. And we need to compute the sum of the reciprocals of sides BC and AC. Maybe I should draw inspiration form another question.

First, the problem says that vectors a and b are orthogonal. Hmm, okay, so that means their dot product is zero. I remember that projections of a vector onto another vector can be found using the formula:

proj_u v = ( (u ⋅ v) / ||u||² ) * u

Where u is the vector onto which we're projecting, and v is the vector we're projecting onto. So, in this case, we're given that the projection of the vector [0, 13] onto a is [6, 4]. Let me write that down:

proj_a [0; 13] = [6; 4]

And we need to find the projection of the same vector [0; 13] onto b, which is orthogonal to a.

Since a and b are orthogonal, they form a basis for the space, right? So any vector can be expressed as a linear combination of a and b. That makes sense because they're orthogonal. So, in this case, since we're dealing with 2D vectors, we can write:

[0; 13] = proj_a [0; 13] + proj_b [0; 13]

Because if a and b are orthogonal, the projections onto them should add up to the original vector. Is that correct? Let me think about that. Yes, because if you project a vector onto two orthogonal subspaces and add the projections, you get the original vector back. That's similar to how in 3D, if you have orthogonal axes, you can decompose a vector into its components along each axis.

So if that's the case, then:

proj_b [0; 13] = [0; 13] - proj_a [0; 13]

Given that proj_a [0; 13] is [6; 4], we can subtract that from [0; 13] to get the projection onto b.

So, let's do that calculation:

proj_b [0; 13] = [0; 13] - [6; 4] = [0 - 6; 13 - 4] = [-6; 9]

Wait, that seems straightforward. Let me just verify if that makes sense.

Since a and b are orthogonal, the projection of the vector onto b should be the leftover part after subtracting the projection onto a. So, if I project [0; 13] onto a, I get [6; 4], and the remaining part is [-6; 9], which should be the projection onto b.

Let me check if I can verify this another way. Maybe by using the formula for projection onto b, since a and b are orthogonal, the projection of v onto b should be zero when you project v onto a, and vice versa.

Let me recall the projection formula for b. It's similar to a:

proj_b [0; 13] = ( ( [0; 13] ⋅ b ) / ||b||² ) * b

But we don't know b yet. However, since a and b are orthogonal, their dot product is zero. So, maybe we can use the fact that the sum of the projections onto a and b should equal the original vector.

But wait, in our earlier step, we used the fact that a and b are orthogonal to decompose the vector into two orthogonal components: one onto a and one onto b. That seems solid.

Let me also consider another approach. The projection of a vector onto a subspace is the closest vector in that subspace to the original vector. Since a and b are orthogonal, the projection onto a subspace spanned by a is just the component along a, and the projection onto the orthogonal complement (spanned by b) is the component along b.

So, if we have:

proj_a v + proj_b v = v

Then, proj_b v = v - proj_a v

Which is exactly what we did earlier. So, that seems correct.

Just to be thorough, let me compute the norms or see if there's any inconsistency.

We have proj_a v = [6; 4]. Let's compute its norm squared:

||proj_a v||² = 6² + 4² = 36 + 16 = 52

So, the norm squared of the projection is 52. The norm squared of [0; 13] is 0² + 13² = 169.

Wait, but if proj_a v is the projection, isn't it scaled? Because the projection formula is ( (v ⋅ a)/||a||² ) a, so the norm squared of proj_a v is ( (v ⋅ a)^2 ) / ||a||².

But in our case, proj_a v = [6; 4], so let's compute the norm squared of that: 6² + 4² = 52 as above. So, ||proj_a v||² = 52.

On the other hand, the norm squared of v is ||v||² = 13² = 169. So, 52 is a fraction of 169, which is 52/169 = 4/13.

So, that means proj_a v is 4/13 times v. Is that the case?

Wait, let me compute (v ⋅ a)/||a||². If proj_a v is [6; 4], then (v ⋅ a) / ||a||² must be 6 / 4 = 3/2? Wait, no, that can't be.

Wait, let me compute the dot product v ⋅ a. Let's denote a as [a1; a2]. Then, proj_a v = ( (v ⋅ a)/||a||² ) a = [6; 4]. So, let's compute (v ⋅ a)/||a||² times a equals [6; 4].

Let me denote (v ⋅ a)/||a||² = k, so k * a = [6; 4]. So, k a1 = 6 and k a2 = 4. Therefore, a1 = 6/k, a2 = 4/k.

Now, the norm squared of a is ||a||² = a1² + a2² = (36)/(k²) + (16)/(k²) = (52)/k².

But we also have that proj_a v = [6; 4] = (k a) = k * [a1; a2] = k * [6/k; 4/k] = [6; 4], which is consistent.

But we don't know k yet. Wait, but we know that a and b are orthogonal. So, maybe we can use that.

Wait, if a and b are orthogonal, then the projection of v onto b is (v ⋅ b)/||b||² * b.

But we don't know b yet. Hmm, perhaps we can use the fact that the projections onto a and b should add up to v.

Wait, I think I might be overcomplicating. Since we have that proj_a v + proj_b v = v, then proj_b v = v - proj_a v. So, as long as v is in the space spanned by a and b, which it is, since we're projecting it onto a and b.

Therefore, proj_b v should be [0; 13] minus [6; 4], which is [-6; 9].

That makes sense, right? Because if I project [0;13] onto a, I get [6;4], and the remaining part is [-6;9], which should be the projection onto b.

Let me just make sure that this is consistent.

Let me check if [6;4] is indeed the projection of [0;13] onto a. 

# Original Response